In [ ]:
import pandas as pd
import requests

# Baixar os dados do link
url = "https://n8n.coinverge.com.br/webhook/precos"
response = requests.get(url)
data = response.json()

# Converter para DataFrame
df = pd.DataFrame(data)

# Exemplo de conversão de tipos de dados
# Ajuste conforme os tipos reais dos dados
for col in df.columns:
    # Tenta converter para numérico, se possível
    df[col] = pd.to_numeric(df[col], errors='ignore')

# Exibe os tipos de dados convertidos
print(df.dtypes)
df.head()

In [ ]:
import numpy as np

# Função para simular a estratégia de stop loss dinâmica
def simular_stop_estrategia(df):
    df = df.sort_values(['simbolo', 'datahora_fechamento']).copy()
    df['stop'] = np.nan
    df['retorno_estrategia'] = np.nan
    df['acumulado_estrategia'] = np.nan
    df['retorno_real'] = (df['ultimo_preco'] - df['preco_abertura']) / df['preco_abertura']
    df['acumulado_real'] = df.groupby('simbolo')['retorno_real'].transform(lambda x: (1 + x).cumprod())


    for simbolo, grupo in df.groupby('simbolo'):
        idx = grupo.index
        stop_vals = []
        retorno_estrategia = []
        acumulado = []
        prev_stop = None
        prev_acumulado = 1

        for i in range(len(grupo)):
            fechamento = grupo.iloc[i]['ultimo_preco']
            minimo = grupo.iloc[i]['preco_minimo']
            abertura = grupo.iloc[i]['preco_abertura']

            # Calcula o stop do próximo dia com base no dia anterior
            if i > 0:
                prev_fechamento = grupo.iloc[i-1]['ultimo_preco']
                prev_minimo = grupo.iloc[i-1]['preco_minimo']
                prev_stop = (prev_fechamento - prev_minimo) / prev_fechamento
            else:
                prev_stop = np.nan

            stop_vals.append(prev_stop)

            # Simula a estratégia
            if np.isnan(prev_stop):
                retorno = (fechamento - abertura) / abertura
            else:
                stop_preco = abertura * (1 - prev_stop)
                if minimo <= stop_preco:
                    # Stop ativado: vende no preço de stop
                    retorno = (stop_preco - abertura) / abertura
                else:
                    # Stop não ativado: vende no fechamento
                    retorno = (fechamento - abertura) / abertura

            retorno_estrategia.append(retorno)
            prev_acumulado *= (1 + retorno)
            acumulado.append(prev_acumulado)

        df.loc[idx, 'stop'] = stop_vals
        df.loc[idx, 'retorno_estrategia'] = retorno_estrategia
        df.loc[idx, 'acumulado_estrategia'] = acumulado

    return df

# Exemplo de uso:
resultado = simular_stop_estrategia(df)
# Exibe as primeiras linhas do resultado para conferência
resultado[['simbolo', 'datahora_fechamento', 'preco_abertura', 'ultimo_preco', 'preco_minimo', 'stop', 'retorno_estrategia', 'acumulado_estrategia', 'retorno_real', 'acumulado_real']].head(20)

In [ ]:
import pandas as pd
import numpy as np
from ta.momentum import RSIIndicator
from ta.trend import EMAIndicator
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

class BacktestSimulador:
    def __init__(self, df, simbolo, params=None):
        self.df = df[df['simbolo'] == simbolo].copy()
        self.simbolo = simbolo
        self.params = params if params else {}
        self.preditores = self.params.get('preditores', [
            'rsi_14', 'mm_exp_21', 'razao_amplitude', 'max_div_min', 'ret_dia_anterior'
        ])
        self.modelo = self.params.get('modelo', DecisionTreeClassifier(random_state=42))
        self.stop_loss = self.params.get('stop_loss', {'tipo': 'fixo', 'valor': 0.05})
        self.proporcao_teste = self.params.get('proporcao_teste', 0.25)
        self.DEBUG = self.params.get('DEBUG', False)
        self._prepara_dados()

    def _prepara_dados(self):
        # Calcula indicadores técnicos
        self.df['rsi_14'] = RSIIndicator(close=self.df['ultimo_preco'], window=14).rsi()
        self.df['mm_exp_21'] = EMAIndicator(close=self.df['ultimo_preco'], window=21).ema_indicator()
        self.df['razao_amplitude'] = (self.df['preco_maximo'] - self.df['preco_minimo']) / self.df['preco_abertura']
        self.df['max_div_min'] = self.df['preco_maximo'] / self.df['preco_minimo']
        self.df['ret_dia_anterior'] = self.df['ultimo_preco'].pct_change()
        self.df['target'] = self.df['variacao_futura_de_preco_binaria']
        self.df = self.df.dropna(subset=self.preditores + ['target'])

    def treina_modelo(self):
        X = self.df[self.preditores]
        y = self.df['target']
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=self.proporcao_teste, random_state=42)
        self.modelo.fit(X_train, y_train)
        y_pred = self.modelo.predict(X_test)
        self.acuracia = accuracy_score(y_test, y_pred)
        if self.DEBUG:
            print(f'Acurácia do modelo para {self.simbolo}: {self.acuracia:.4f}')

    def simula_estrategia(self):
        # Exemplo de simulação simples com stop loss fixo
        df_sim = self.df.copy()
        df_sim['sinal'] = self.modelo.predict(df_sim[self.preditores])
        df_sim['retorno'] = 0.0
        stop_valor = self.stop_loss['valor']
        for i in range(1, len(df_sim)):
            if df_sim.iloc[i]['sinal'] == 1:
                abertura = df_sim.iloc[i]['preco_abertura']
                fechamento = df_sim.iloc[i]['ultimo_preco']
                minimo = df_sim.iloc[i]['preco_minimo']
                stop_preco = abertura * (1 - stop_valor)
                if minimo <= stop_preco:
                    retorno = (stop_preco - abertura) / abertura
                else:
                    retorno = (fechamento - abertura) / abertura
                df_sim.iat[i, df_sim.columns.get_loc('retorno')] = retorno
        df_sim['acumulado'] = (1 + df_sim['retorno']).cumprod()
        self.resultado = df_sim

    def getRetornoMedioHorario(self):
        a = self.df.copy()
        a = a.reset_index()[['ultimo_preco', 'datahora_fechamento']]
        a['hora_inicio'] = pd.to_datetime(a['datahora_fechamento']).dt.hour
        a['retorno_abertura_fechamento'] = a['ultimo_preco'].pct_change()
        a = a.dropna()
        return a.groupby('hora_inicio')['retorno_abertura_fechamento'].mean()

# Exemplo de uso:
# sim = BacktestSimulador(df, simbolo, params={'DEBUG': True})
# sim.treina_modelo()
# sim.simula_estrategia()
# print(sim.resultado.head())

In [ ]:
# Classe de backtest com stop dinâmico e campos vazios na primeira linha de cada ativo
class BacktestStopDinamico:
    def __init__(self, df):
        self.df = df.copy()
        self.resultados = {}

    def simular(self):
        for simbolo, grupo in self.df.groupby('simbolo'):
            grupo = grupo.sort_values('datahora_fechamento').copy()
            grupo['stop'] = np.nan
            grupo['retorno_estrategia'] = np.nan
            grupo['acumulado_estrategia'] = np.nan
            grupo['variacao_acumulado_max_desceu'] = np.nan

            prev_stop = None
            prev_acumulado = 1
            max_acumulado = 1
            stop_vals = []
            retorno_estrategia = []
            acumulado = []
            variacao_max_desceu = []

            for i in range(len(grupo)):
                if i == 0:
                    # Primeira linha: campos vazios
                    stop_vals.append(np.nan)
                    retorno_estrategia.append(np.nan)
                    acumulado.append(np.nan)
                    variacao_max_desceu.append(np.nan)
                    continue

                fechamento = grupo.iloc[i]['ultimo_preco']
                minimo = grupo.iloc[i]['preco_minimo']
                abertura = grupo.iloc[i]['preco_abertura']

                prev_fechamento = grupo.iloc[i-1]['ultimo_preco']
                prev_minimo = grupo.iloc[i-1]['preco_minimo']
                prev_stop = (prev_fechamento - prev_minimo) / prev_fechamento
                stop_vals.append(prev_stop)

                stop_preco = abertura * (1 - prev_stop)
                if minimo <= stop_preco:
                    retorno = (stop_preco - abertura) / abertura
                else:
                    retorno = (fechamento - abertura) / abertura

                retorno_estrategia.append(retorno)
                prev_acumulado *= (1 + retorno)
                acumulado.append(prev_acumulado)
                max_acumulado = max(max_acumulado, prev_acumulado)
                variacao_max_desceu.append((prev_acumulado - max_acumulado) / max_acumulado)

            grupo['stop'] = stop_vals
            grupo['retorno_estrategia'] = retorno_estrategia
            grupo['acumulado_estrategia'] = acumulado
            grupo['variacao_acumulado_max_desceu'] = variacao_max_desceu

            self.resultados[simbolo] = grupo

    def plot_tabelas(self):
        for simbolo, resultado in self.resultados.items():
            print(f"Tabela de resultados para {simbolo}:")
            display(resultado[['datahora_fechamento', 'preco_abertura', 'ultimo_preco', 'preco_minimo', 'stop', 'retorno_estrategia', 'acumulado_estrategia', 'variacao_acumulado_max_desceu']].head(20))

# Exemplo de uso:
backtest = BacktestStopDinamico(df)
backtest.simular()
backtest.plot_tabelas()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display  # Para display em Jupyter

class BacktestStopDinamico:
    def __init__(self, df):
        self.df = df.copy()
        self.resultados = {}

    def simular(self):
        for simbolo, grupo in self.df.groupby('simbolo'):
            if len(grupo) < 24:
                continue  # Remove ativos com menos de 24 registros
            
            grupo = grupo.sort_values('datahora_fechamento').copy()
            grupo['stop'] = np.nan
            grupo['retorno_estrategia'] = np.nan
            grupo['acumulado_estrategia'] = np.nan
            grupo['variacao_acumulado_max_desceu'] = np.nan

            prev_stop = None
            prev_acumulado = 1
            max_acumulado = 1
            stop_vals = []
            retorno_estrategia = []
            acumulado = []
            variacao_max_desceu = []

            for i in range(len(grupo)):
                if i == 0:
                    # Primeira linha: campos vazios
                    stop_vals.append(np.nan)
                    retorno_estrategia.append(np.nan)
                    acumulado.append(np.nan)
                    variacao_max_desceu.append(np.nan)
                    continue

                fechamento = grupo.iloc[i]['ultimo_preco']
                minimo = grupo.iloc[i]['preco_minimo']
                abertura = grupo.iloc[i]['preco_abertura']

                prev_fechamento = grupo.iloc[i-1]['ultimo_preco']
                prev_minimo = grupo.iloc[i-1]['preco_minimo']
                prev_stop = (prev_fechamento - prev_minimo) / prev_fechamento
                stop_vals.append(prev_stop)

                stop_preco = abertura * (1 - prev_stop)
                if minimo <= stop_preco:
                    retorno = (stop_preco - abertura) / abertura
                else:
                    retorno = (fechamento - abertura) / abertura

                retorno_estrategia.append(retorno)
                prev_acumulado *= (1 + retorno)
                acumulado.append(prev_acumulado)
                max_acumulado = max(max_acumulado, prev_acumulado)
                variacao_max_desceu.append((prev_acumulado - max_acumulado) / max_acumulado)

            grupo['stop'] = stop_vals
            grupo['retorno_estrategia'] = retorno_estrategia
            grupo['acumulado_estrategia'] = acumulado
            grupo['variacao_acumulado_max_desceu'] = variacao_max_desceu

            self.resultados[simbolo] = grupo

    def plot_tabelas(self):
        # Limitar o número de ativos plotados para evitar sobrecarga de desempenho (ex: primeiros 5)
        simbolos_para_plotar = list(self.resultados.keys())[:5]  # Ajuste o número conforme necessário
        
        for simbolo in self.resultados.keys():
            resultado = self.resultados[simbolo]
            print(f"Tabela de resultados para {simbolo}:")
            display(resultado[['datahora_fechamento', 'preco_abertura', 'ultimo_preco', 'preco_minimo', 'stop', 'retorno_estrategia', 'acumulado_estrategia', 'variacao_acumulado_max_desceu']].head(20))
            
            # Plotar gráfico da variação do stop por ativo, mas apenas se for um dos selecionados e com poucos registros (ex: head(50))
            if simbolo in simbolos_para_plotar:
                df_plot = resultado.head(50)  # Limitar a 50 registros para desempenho
                plt.figure(figsize=(10, 6))
                plt.plot(df_plot['datahora_fechamento'], df_plot['stop'], marker='o', linestyle='-', color='b')
                plt.title(f'Variação do Stop para {simbolo}')
                plt.xlabel('Data/Hora de Fechamento')
                plt.ylabel('Valor do Stop')
                plt.xticks(rotation=45)
                plt.grid(True)
                plt.tight_layout()
                plt.show()

# Exemplo de uso:
# backtest = BacktestStopDinamico(df)
# backtest.simular()
# backtest.plot_tabelas()


In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display  # Para display em Jupyter

class BacktestStopDinamico:
    def __init__(self, df):
        self.df = df.copy()
        self.resultados = {}

    def simular(self):
        for simbolo, grupo in self.df.groupby('simbolo'):
            if len(grupo) < 24:
                continue  # Remove ativos com menos de 24 registros
            
            grupo = grupo.sort_values('datahora_fechamento').copy()
            
            # Colunas para o ativo
            grupo['var'] = np.nan
            grupo['acumulado'] = np.nan
            grupo['max_desceu'] = np.nan
            
            # Colunas da estratégia
            grupo['stop'] = np.nan
            grupo['retorno_estrategia'] = np.nan
            grupo['acumulado_estrategia'] = np.nan

            prev_stop = None
            prev_acumulado = 1
            max_acumulado = 1
            stop_vals = []
            retorno_estrategia = []
            acumulado_estrategia = []
            var_vals = []
            acumulado_ativo = []
            max_desceu_vals = []
            max_acumulado_ativo = 1

            for i in range(len(grupo)):
                if i == 0:
                    # Primeira linha: campos vazios
                    stop_vals.append(np.nan)
                    retorno_estrategia.append(np.nan)
                    acumulado_estrategia.append(np.nan)
                    var_vals.append(np.nan)
                    acumulado_ativo.append(np.nan)
                    max_desceu_vals.append(np.nan)
                    continue

                fechamento = grupo.iloc[i]['ultimo_preco']
                minimo = grupo.iloc[i]['preco_minimo']
                maximo = grupo.iloc[i].get('preco_maximo', np.nan)  # Assumindo que 'preco_maximo' existe; caso contrário, NaN
                abertura = grupo.iloc[i]['preco_abertura']

                # Cálculos para o ativo
                var = (fechamento - abertura) / abertura
                var_vals.append(var)
                if i == 1:
                    acumulado_ativo_val = 1 + var
                else:
                    acumulado_ativo_val = acumulado_ativo[-1] * (1 + var)
                acumulado_ativo.append(acumulado_ativo_val)
                max_acumulado_ativo = max(max_acumulado_ativo, acumulado_ativo_val)
                max_desceu = (acumulado_ativo_val - max_acumulado_ativo) / max_acumulado_ativo
                max_desceu_vals.append(max_desceu)

                # Cálculos para stop e estratégia
                prev_fechamento = grupo.iloc[i-1]['ultimo_preco']
                prev_minimo = grupo.iloc[i-1]['preco_minimo']
                prev_stop = (prev_fechamento - prev_minimo) / prev_fechamento
                stop_vals.append(prev_stop)

                stop_preco = abertura * (1 - prev_stop)
                
                # Nova lógica para retorno_estrategia
                max_desceu_periodo = (minimo - abertura) / abertura  # Drawdown intra-período como "max que desceu"
                if max_desceu_periodo < prev_stop:
                    # Se max_desceu < stop, aplica a penalidade (stop - 0.0575% - 0.1%)
                    retorno = prev_stop - 0.000575 - 0.001
                else:
                    # Caso contrário, usa a variação do fechamento (var)
                    retorno = var

                retorno_estrategia.append(retorno)
                prev_acumulado *= (1 + retorno)
                acumulado_estrategia.append(prev_acumulado)
                max_acumulado = max(max_acumulado, prev_acumulado)

            # Atribui as colunas ao grupo
            grupo['var'] = var_vals
            grupo['acumulado'] = acumulado_ativo
            grupo['max_desceu'] = max_desceu_vals
            grupo['stop'] = stop_vals
            grupo['retorno_estrategia'] = retorno_estrategia
            grupo['acumulado_estrategia'] = acumulado_estrategia

            self.resultados[simbolo] = grupo

    def plot_tabelas(self):
        # Limitar o número de ativos para plotar gráficos (somente os 2 primeiros)
        simbolos_para_plotar = list(self.resultados.keys())[:2]  # Apenas os 2 primeiros
        
        for simbolo in self.resultados.keys():
            resultado = self.resultados[simbolo]
            print(f"Tabela de resultados para {simbolo}:")
            
            # Renomear colunas para matching o exemplo (cabeçalhos desejados)
            tabela_display = resultado[['datahora_fechamento', 'preco_abertura', 'ultimo_preco', 'preco_minimo', 'preco_maximo', 'var', 'acumulado', 'max_desceu', 'stop', 'retorno_estrategia', 'acumulado_estrategia']].copy()
            tabela_display = tabela_display.rename(columns={
                'datahora_fechamento': 'DATA',
                'preco_abertura': 'ABERTURA',
                'ultimo_preco': 'FECHAMENTO',
                'preco_minimo': 'MIN',
                'preco_maximo': 'MAX',
                'max_desceu': 'max-desceu',
                'retorno_estrategia': 'retorno-estrategia',
                'acumulado_estrategia': 'acumulado-estrategia'
            })
            resultado[['acumulado_estrategia', 'acumulado']].reset_index(drop=True).plot()
            
            # Plotar gráfico da variação do stop por ativo, mas apenas se for um dos 2 primeiros e com poucos registros (ex: head(50))
            if simbolo in simbolos_para_plotar:
                df_plot = resultado.head(50)
                plt.figure(figsize=(10, 6))
                plt.plot(df_plot['datahora_fechamento'], df_plot['stop'], marker='o', linestyle='-', color='b')
                plt.title(f'Variação do Stop para {simbolo}')
                plt.xlabel('Data/Hora de Fechamento')
                plt.ylabel('Valor do Stop')
                plt.xticks(rotation=45)
                plt.grid(True)
                plt.tight_layout()
                plt.show()

# Exemplo de uso:
# backtest = BacktestStopDinamico(df)
# backtest.simular()
backtest.plot_tabelas()


Tabela de resultados para 1000CATUSDT:


,datahora_fechamento,preco_abertura,ultimo_preco,preco_minimo,stop,retorno_estrategia,acumulado_estrategia,variacao_acumulado_max_desceu
0,2025-07-28T09:00:56.000Z,0.00822,0.01016,0.00813,NaN,NaN,NaN,NaN
1,2025-07-28T12:00:56.000Z,0.00821,0.01061,0.00814,0.199803,0.292326,1.292326,0.0
2,2025-07-28T13:00:56.000Z,0.00818,0.01277,0.00818,0.232799,0.561125,2.017483,0.0
3,2025-07-28T14:00:56.000Z,0.00835,0.01252,0.00827,0.359436,0.499401,3.025016,0.0
4,2025-07-28T16:00:55.000Z,0.00845,0.01245,0.00827,0.339457,0.473373,4.456976,0.0


Tabela de resultados para 1000CHEEMSUSDT:


,datahora_fechamento,preco_abertura,ultimo_preco,preco_minimo,stop,retorno_estrategia,acumulado_estrategia,variacao_acumulado_max_desceu
38,2025-07-23T17:15:22.000Z,0.001364,0.001328,0.001322,NaN,NaN,NaN,NaN
73,2025-07-23T18:30:23.000Z,0.001370,0.001323,0.001317,0.004518,-0.004518,0.995482,-0.004518
190,2025-07-23T19:00:17.000Z,0.001363,0.001332,0.001317,0.004535,-0.004535,0.990967,-0.009033
191,2025-07-23T19:15:23.000Z,0.001373,0.001322,0.001317,0.011261,-0.011261,0.979808,-0.020192
39,2025-07-23T19:45:23.000Z,0.001366,0.001311,0.001300,0.003782,-0.003782,0.976102,-0.023898
192,2025-07-23T20:00:23.000Z,0.001364,0.001307,0.001300,0.008391,-0.008391,0.967912,-0.032088
193,2025-07-23T20:30:23.000Z,0.001358,0.001304,0.001296,0.005356,-0.005356,0.962728,-0.037272
40,2025-07-23T21:00:23.000Z,0.001363,0.001324,0.001296,0.006135,-0.006135,0.956822,-0.043178
41,2025-07-23T21:19:23.000Z,0.001364,0.001317,0.001296,0.021148,-0.021148,0.936587,-0.063413
42,2025-07-23T21:24:27.000Z,0.001368,0.001323,0.001296,0.015945,-0.015945,0.921653,-0.078347


Tabela de resultados para 1INCHUSDT:


...

,datahora_fechamento,preco_abertura,ultimo_preco,preco_minimo,stop,retorno_estrategia,acumulado_estrategia,variacao_acumulado_max_desceu
28509,2025-07-23T15:15:23.000Z,0.002028,0.002008,0.001975,NaN,NaN,NaN,NaN
19792,2025-07-23T15:30:23.000Z,0.002022,0.002008,0.001975,0.016434,-0.016434,0.983566,-0.016434
19793,2025-07-23T16:00:23.000Z,0.002022,0.001945,0.001939,0.016434,-0.016434,0.967402,-0.032598
19794,2025-07-23T16:15:23.000Z,0.002034,0.001952,0.001927,0.003085,-0.003085,0.964417,-0.035583
19795,2025-07-23T17:00:23.000Z,0.002011,0.001916,0.001888,0.012807,-0.012807,0.952066,-0.047934
19796,2025-07-23T17:15:23.000Z,0.002013,0.001928,0.001888,0.014614,-0.014614,0.938152,-0.061848
19797,2025-07-23T17:30:21.000Z,0.002006,0.001947,0.001888,0.020747,-0.020747,0.918689,-0.081311
19798,2025-07-23T17:45:23.000Z,0.002026,0.001931,0.001888,0.030303,-0.030303,0.890850,-0.109150
19799,2025-07-23T18:00:23.000Z,0.002036,0.001922,0.001888,0.022268,-0.022268,0.871012,-0.128988
19800,2025-07-23T18:45:23.000Z,0.002027,0.001937,0.001888,0.017690,-0.017690,0.855604,-0.144396


Tabela de resultados para METISUSDT:


,datahora_fechamento,preco_abertura,ultimo_preco,preco_minimo,stop,retorno_estrategia,acumulado_estrategia,variacao_acumulado_max_desceu
28874,2025-07-23T15:15:20.000Z,19.00,18.68,18.41,NaN,NaN,NaN,NaN
28875,2025-07-23T15:30:16.000Z,18.96,18.60,18.41,0.014454,-0.014454,0.985546,-0.014454
27721,2025-07-23T16:00:23.000Z,18.93,18.15,18.14,0.010215,-0.010215,0.975479,-0.024521
27722,2025-07-23T16:15:21.000Z,18.99,18.22,18.05,0.000551,-0.000551,0.974941,-0.025059
27723,2025-07-23T16:45:23.000Z,18.92,17.86,17.73,0.009330,-0.009330,0.965845,-0.034155
27724,2025-07-23T17:00:23.000Z,18.68,17.71,17.26,0.007279,-0.007279,0.958814,-0.041186
27725,2025-07-23T17:15:23.000Z,18.79,17.76,17.26,0.025409,-0.025409,0.934451,-0.065549
21257,2025-07-23T17:30:23.000Z,18.68,17.91,17.26,0.028153,-0.028153,0.908144,-0.091856
21258,2025-07-23T18:00:23.000Z,19.22,17.81,17.26,0.036293,-0.036293,0.875185,-0.124815
19923,2025-07-23T18:45:23.000Z,19.17,17.86,17.26,0.030882,-0.030882,0.848158,-0.151842


Tabela de resultados para MINAUSDT:


KeyboardInterrupt: 